# Lab 07: Advanced Conversation Management

**Course:** Generative AI for Banking Sector  
**Institution:** Banco Nacional de Costa Rica (BNCR)  
**Instructor:** Manuela Larrea  
**Duration:** 3 hours

---

## Learning Objectives

By the end of this lab, you will be able to:

1. Build stateful conversational AI systems
2. Implement conversation memory and context management
3. Handle multi-turn dialogues with complex intent tracking
4. Create conversation summarization for long interactions
5. Implement conversation routing and escalation logic
6. Build a production-ready banking chatbot
7. Monitor and log conversations for quality assurance

---

## Azure Infrastructure

```
╔══════════════════════════════════════════════════════════════════════════╗
║                  LAB 07 - AZURE INFRASTRUCTURE                           ║
╚══════════════════════════════════════════════════════════════════════════╝

Same as Lab 06 - Focus on conversation patterns and state management
```

In [ ]:
import os
import sys
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
from datetime import datetime
from typing import List, Dict, Optional

sys.path.append('../../utils')
from azure_openai_helper import AzureOpenAIClient

load_dotenv()
client = AzureOpenAIClient()

print("✓ Setup complete")

## Part 1: Conversation State Management

A production chatbot needs to track:
- Conversation history
- User context (name, account type, preferences)
- Current intent and state
- Pending actions
- Escalation flags

In [ ]:
class ConversationState:
    """Manages conversation state and context"""
    
    def __init__(self, user_id: str):
        self.user_id = user_id
        self.messages = []
        self.context = {}
        self.current_intent = None
        self.pending_actions = []
        self.escalation_needed = False
        self.sentiment_history = []
        self.created_at = datetime.now()
        self.last_updated = datetime.now()
    
    def add_message(self, role: str, content: str):
        """Add a message to conversation history"""
        self.messages.append({
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        })
        self.last_updated = datetime.now()
    
    def set_context(self, key: str, value):
        """Set context variable"""
        self.context[key] = value
    
    def get_context(self, key: str, default=None):
        """Get context variable"""
        return self.context.get(key, default)
    
    def get_messages_for_api(self) -> List[Dict]:
        """Format messages for OpenAI API"""
        return [{"role": m["role"], "content": m["content"]} 
                for m in self.messages]
    
    def to_dict(self) -> Dict:
        """Serialize state to dictionary"""
        return {
            "user_id": self.user_id,
            "messages": self.messages,
            "context": self.context,
            "current_intent": self.current_intent,
            "pending_actions": self.pending_actions,
            "escalation_needed": self.escalation_needed,
            "created_at": self.created_at.isoformat(),
            "last_updated": self.last_updated.isoformat()
        }

# Test the state manager
state = ConversationState("user_12345")
state.add_message("system", "You are a helpful banking assistant.")
state.add_message("user", "I want to open a savings account.")
state.set_context("intent", "account_opening")
state.set_context("product_interest", "savings_account")

print("Conversation State:")
print(json.dumps(state.to_dict(), indent=2))

## Part 2: Advanced Banking Chatbot

Let's build a sophisticated chatbot with:
- Intent detection
- Context awareness
- Multi-turn conversations
- Slot filling (collecting required information)
- Escalation logic

In [ ]:
class AdvancedBankingChatbot:
    """Production-ready banking chatbot with state management"""
    
    SYSTEM_PROMPT = """You are an intelligent banking assistant for BNCR with these capabilities:

CORE FUNCTIONS:
- Answer questions about products and services
- Guide customers through processes (account opening, loan applications, etc.)
- Provide personalized recommendations
- Detect and escalate urgent issues

CONVERSATION GUIDELINES:
1. Be professional, friendly, and helpful
2. Ask clarifying questions when needed
3. Remember context from previous messages
4. Collect required information step-by-step
5. Confirm understanding before taking actions
6. Escalate to human agent when:
   - Customer is frustrated or angry
   - Issue is complex or requires manual intervention
   - Security concerns arise
   - You cannot help with the request

LIMITATIONS:
- Cannot access real account information
- Cannot perform actual transactions
- Cannot provide personalized financial advice
- Cannot discuss competitor banks

Always maintain context and refer to information shared earlier in the conversation."""
    
    def __init__(self, client: AzureOpenAIClient):
        self.client = client
        self.sessions = {}  # Store multiple user sessions
    
    def get_or_create_session(self, user_id: str) -> ConversationState:
        """Get existing session or create new one"""
        if user_id not in self.sessions:
            state = ConversationState(user_id)
            state.add_message("system", self.SYSTEM_PROMPT)
            self.sessions[user_id] = state
        return self.sessions[user_id]
    
    def chat(self, user_id: str, message: str) -> Dict:
        """Process user message and return response with metadata"""
        state = self.get_or_create_session(user_id)
        
        # Add user message
        state.add_message("user", message)
        
        # Detect intent and sentiment (simplified)
        intent = self._detect_intent(message)
        sentiment = self._detect_sentiment(message)
        
        state.current_intent = intent
        state.sentiment_history.append(sentiment)
        
        # Check if escalation needed
        if sentiment == "negative" and len([s for s in state.sentiment_history[-3:] if s == "negative"]) >= 2:
            state.escalation_needed = True
        
        # Get AI response
        messages = state.get_messages_for_api()
        response_text = self.client.chat_completion(
            messages,
            temperature=0.7,
            max_tokens=300
        )
        
        # Add assistant response
        state.add_message("assistant", response_text)
        
        return {
            "response": response_text,
            "intent": intent,
            "sentiment": sentiment,
            "escalation_needed": state.escalation_needed,
            "conversation_length": len(state.messages)
        }
    
    def _detect_intent(self, message: str) -> str:
        """Simple intent detection (in production, use a classifier)"""
        message_lower = message.lower()
        
        if any(word in message_lower for word in ["open", "create", "new account"]):
            return "account_opening"
        elif any(word in message_lower for word in ["loan", "credit", "borrow"]):
            return "loan_inquiry"
        elif any(word in message_lower for word in ["card", "credit card", "debit card"]):
            return "card_inquiry"
        elif any(word in message_lower for word in ["transfer", "send money", "payment"]):
            return "transfer"
        elif any(word in message_lower for word in ["problem", "issue", "not working", "error"]):
            return "technical_support"
        else:
            return "general_inquiry"
    
    def _detect_sentiment(self, message: str) -> str:
        """Simple sentiment detection"""
        message_lower = message.lower()
        
        negative_words = ["frustrated", "angry", "terrible", "awful", "hate", 
                         "worst", "disappointed", "unacceptable"]
        positive_words = ["great", "excellent", "love", "perfect", "amazing", 
                         "wonderful", "thank", "helpful"]
        
        if any(word in message_lower for word in negative_words):
            return "negative"
        elif any(word in message_lower for word in positive_words):
            return "positive"
        else:
            return "neutral"
    
    def get_conversation_summary(self, user_id: str) -> str:
        """Generate conversation summary"""
        if user_id not in self.sessions:
            return "No conversation found."
        
        state = self.sessions[user_id]
        
        # Create summary prompt
        conversation_text = "\n".join([
            f"{m['role'].upper()}: {m['content']}" 
            for m in state.messages if m['role'] != 'system'
        ])
        
        summary_prompt = f"""Summarize this banking customer conversation:

{conversation_text}

Provide:
1. Main topic/intent
2. Key information shared
3. Current status
4. Next steps (if any)

Keep it concise (under 100 words)."""
        
        messages = [
            {"role": "system", "content": "You are a conversation summarizer."},
            {"role": "user", "content": summary_prompt}
        ]
        
        summary = self.client.chat_completion(messages, temperature=0.3, max_tokens=200)
        return summary

# Initialize chatbot
chatbot = AdvancedBankingChatbot(client)
print("✓ Advanced Banking Chatbot initialized")

## Part 3: Testing Multi-Turn Conversations

In [ ]:
# Simulate a multi-turn conversation
user_id = "customer_001"

conversation = [
    "Hello, I'm interested in opening a savings account.",
    "I'm 25 years old and I work as a teacher.",
    "I can deposit about $300 per month.",
    "What interest rate do you offer?",
    "That sounds good. What documents do I need?"
]

for i, user_message in enumerate(conversation, 1):
    print(f"\n{'='*80}")
    print(f"Turn {i}")
    print(f"{'='*80}")
    print(f"\nUser: {user_message}")
    
    result = chatbot.chat(user_id, user_message)
    
    print(f"\nAssistant: {result['response']}")
    print(f"\nMetadata:")
    print(f"  Intent: {result['intent']}")
    print(f"  Sentiment: {result['sentiment']}")
    print(f"  Escalation needed: {result['escalation_needed']}")
    print(f"  Messages in conversation: {result['conversation_length']}")

## Part 4: Conversation Summarization

In [ ]:
# Get conversation summary
print("\n" + "="*80)
print("CONVERSATION SUMMARY")
print("="*80 + "\n")

summary = chatbot.get_conversation_summary(user_id)
print(summary)

## Part 5: Handling Escalation Scenarios

In [ ]:
# Test escalation with frustrated customer
user_id_2 = "customer_002"

frustrated_conversation = [
    "Your app is not working again!",
    "This is the third time this week. I'm very frustrated.",
    "I need to pay my bills and I can't access my account. This is unacceptable!"
]

print("\nTesting Escalation Scenario:")
print("="*80)

for i, message in enumerate(frustrated_conversation, 1):
    print(f"\nTurn {i}")
    print(f"User: {message}")
    
    result = chatbot.chat(user_id_2, message)
    
    print(f"Assistant: {result['response']}")
    print(f"Sentiment: {result['sentiment']}")
    print(f"⚠️ ESCALATION NEEDED: {result['escalation_needed']}")
    
    if result['escalation_needed']:
        print("\n🚨 ALERT: Customer should be transferred to human agent!")
        break

## Part 6: Conversation Logging and Analytics

In [ ]:
class ConversationLogger:
    """Log and analyze conversations for quality assurance"""
    
    def __init__(self):
        self.logs = []
    
    def log_interaction(self, user_id: str, user_message: str, 
                       bot_response: str, metadata: Dict):
        """Log a single interaction"""
        self.logs.append({
            "timestamp": datetime.now().isoformat(),
            "user_id": user_id,
            "user_message": user_message,
            "bot_response": bot_response,
            "intent": metadata.get("intent"),
            "sentiment": metadata.get("sentiment"),
            "escalation_needed": metadata.get("escalation_needed", False)
        })
    
    def get_analytics(self) -> Dict:
        """Generate analytics from logs"""
        if not self.logs:
            return {"error": "No logs available"}
        
        total_interactions = len(self.logs)
        
        # Intent distribution
        intents = {}
        for log in self.logs:
            intent = log.get("intent", "unknown")
            intents[intent] = intents.get(intent, 0) + 1
        
        # Sentiment distribution
        sentiments = {}
        for log in self.logs:
            sentiment = log.get("sentiment", "unknown")
            sentiments[sentiment] = sentiments.get(sentiment, 0) + 1
        
        # Escalation rate
        escalations = sum(1 for log in self.logs if log.get("escalation_needed"))
        escalation_rate = (escalations / total_interactions) * 100
        
        return {
            "total_interactions": total_interactions,
            "intent_distribution": intents,
            "sentiment_distribution": sentiments,
            "escalations": escalations,
            "escalation_rate": f"{escalation_rate:.1f}%"
        }
    
    def export_logs(self, filename: str):
        """Export logs to JSON file"""
        with open(filename, 'w') as f:
            json.dump(self.logs, f, indent=2)
        print(f"✓ Logs exported to {filename}")

# Test logger
logger = ConversationLogger()

# Log some sample interactions
logger.log_interaction("user_001", "Hello", "Hi! How can I help?", 
                      {"intent": "greeting", "sentiment": "neutral"})
logger.log_interaction("user_001", "I want a loan", "I can help with that.", 
                      {"intent": "loan_inquiry", "sentiment": "neutral"})
logger.log_interaction("user_002", "Your app is broken!", "I apologize...", 
                      {"intent": "technical_support", "sentiment": "negative", "escalation_needed": True})

# Get analytics
print("\nConversation Analytics:")
print(json.dumps(logger.get_analytics(), indent=2))

## 🎯 Practical Exercise 1: Loan Application Assistant

Build a conversational assistant that guides users through a loan application.

Requirements:
- Collect: loan amount, purpose, income, employment status
- Validate inputs
- Provide preliminary eligibility assessment
- Generate application summary
- Handle incomplete information gracefully

In [ ]:
# TODO: Create loan application assistant
# Your code here:


## 🎯 Practical Exercise 2: Intelligent Routing System

Create a system that routes conversations to appropriate departments.

Departments:
- Customer Service (general inquiries)
- Technical Support (app/website issues)
- Fraud Department (suspicious activity)
- Loan Department (loan applications)
- VIP Service (high-value customers)

Include urgency scoring and escalation logic.

In [ ]:
# TODO: Create routing system
# Your code here:


## 🎯 Practical Exercise 3: Conversation Quality Metrics

Implement a system to measure conversation quality.

Metrics to track:
- Average conversation length
- Resolution rate (did the bot solve the issue?)
- Customer satisfaction (based on sentiment)
- Response time
- Escalation rate
- Intent accuracy

Create a dashboard summary.

In [ ]:
# TODO: Implement quality metrics system
# Your code here:


## Summary

In this lab, you learned how to build production-ready conversational AI systems with:

- **State Management**: Tracking conversation history and context
- **Intent Detection**: Understanding what users want
- **Sentiment Analysis**: Detecting customer emotions
- **Escalation Logic**: Knowing when to involve humans
- **Conversation Summarization**: Creating concise summaries
- **Logging and Analytics**: Monitoring performance

### Best Practices:

1. **Always maintain context** across conversation turns
2. **Detect and respond to sentiment** changes
3. **Have clear escalation criteria** for human handoff
4. **Log everything** for quality assurance and improvement
5. **Test with realistic scenarios** including edge cases
6. **Monitor metrics** to continuously improve

### Next Lab:

Lab 08: Function Calling - Learn how to integrate external tools and APIs with your chatbot.

---

**Instructor:** Manuela Larrea | manuela.larrea@idataglobal.com